In [42]:
import os, sys
import openai
import pinecone
import pandas as pd
from dotenv import dotenv_values
from llama_hub.confluence.base import ConfluenceReader
from langchain.chat_models import ChatOpenAI
from llama_index import LLMPredictor, ServiceContext, GPTVectorStoreIndex
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores import PineconeVectorStore
from llama_index import set_global_service_context

config = dotenv_values(".env")

openai.organization = config["ORG_ID"]
openai.api_key = config["OPENAI_API_KEY"]

# os.environ["CONFLUENCE_API_TOKEN"] = config["CONFLUENCE_API_TOKEN"]
# os.environ["CONFLUENCE_USERNAME"] = config["CONFLUENCE_USERNAME"]

# set the size of the context window of the LLM. Typically this is set automatically with the model metadata. We can also explicitly override via this parameter for additional control
context_window = 4096
# set number of output tokens from the LLM. Typically this is set automatically with the model metadata. It doesn't limit the model output, but affects the amount of “space” we save for the output, when computing available context window size for packing text from retrieved Nodes
num_output = 512

#LLMPredictor is a wrapper class around LangChain's LLMChain that allows easy integration into LlamaIndex
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0.2, model_name="gpt-3.5-turbo", max_tokens=num_output))

#constructs service_context
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, 
                                                context_window=context_window,
                                                num_output=num_output)

set_global_service_context(service_context)

# init pinecone
pinecone.init(api_key=config['PINECONE_API_KEY'], environment="asia-southeast1-gcp-free")

def data_ingestion_indexing():

    # Confluence base url and space key
    base_url = "https://navihq.atlassian.net/wiki/"
    space_key = "IN"
    

    # construct ConfluenceReader and load data from Confluence
    loader = ConfluenceReader(base_url=base_url)
    documents = loader.load_data(space_key=space_key, page_ids=[], include_attachments=True)
    # set pinecone index
    pinecone_index = pinecone.Index("confluence-index")
    # build the PineconeVectorStore and GPTVectorStoreIndex
    vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = GPTVectorStoreIndex.from_documents(documents, storage_context=storage_context)
    return index

index = data_ingestion_indexing()

Upserted vectors:   0%|          | 0/274 [3:37:26<?, ?it/s]


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [44]:
query_suffix = ".Use bullet points wherever applicable. Please always cite sources along with your answer."

def data_querying(input_text):
    result = index.as_query_engine(service_context=service_context).query(input_text + query_suffix)
    return result


print(data_querying("How to take a heap dump in production"))


There are several ways to take a heap dump in production:

1. Using the Toolbox Pod (Nonprod): Exec into the toolbox pod and run the command `curl http://payments-navi-service.dev-critical:4001/actuator/heapdump -O`. Use Pod IP if you want to take specific pods heap dump.

2. Using op-task: Op-tasks setup can be used to get a heap dump for a service. Once the op-task pipeline passes, an email is sent to the user who ran the pipeline with links to download artifacts and log files.

3. Using ephemeral containers: Use the command to create a copy of the pod. This copy will contain an injected container that has the tools to take the heap dump. Once the pod is ready, run the command to exec into the created pod. Run `ps aux` command and get the process id of the java process. Use the command to create a heap dump inside the pod. Heapdump file should be created at path `/proc/<pid>/root/tmp/`. Run the command to copy the heap dump file outside the pod.

4. Automatic Heap Dump on Java OOM: T